### Import library

In [30]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    CV branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

' Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP\n    Goal is to predict if mutation is SNP or PD\n    CV branch\n    \n    Total samples: 3368\n    2254 PD samples\n    1111 SNP samples\n'

In [36]:
""" Imports the required libraries and packages """

import pandas as pd  #Import for data manipulation in dataframes
import numpy as np  #Array manipulation and calculates mean

import random as rd
import time

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )
from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Cross validation to improve hyperparameters
    RandomizedSearchCV,
    KFold,
    StratifiedKFold, # K-fold CV
    GroupKFold,
    StratifiedGroupKFold
        )

from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each
np.set_printoptions(threshold=np.inf, precision=3) #full array printing

### Clean dataset with pandas

In [41]:
def Clean_data(file):
    """ Input:      file        The dataset to read

        Returns:    Output      Cleaned dataframe with numeric values for class

        Create, clean and convert dataset E2.csv to PD dataframe. Sorts alphabetically to order proteins, removes blank spaces
        and applies "One Hot Encoding" to convert classes (PD/SNP) to 1/0
    """
    df = pd.read_csv('E2.csv')

    #Remove unrequired column, replace blank spaces, reset index to run from 0
    df.dropna(inplace = True) #drop rows with missing values
    df.replace(' ', '_', regex=True, inplace=True)
    df.reset_index(drop=True, inplace = True)

#     Input = df.drop('dataset', axis =1)
    Output_encoded = pd.get_dummies(df, columns=['dataset']) #Encode the PD and SNP columns
    Output = Output_encoded.drop(['dataset_snp'],axis = 1)
#     Output = Output_encoded['dataset_pd'].copy().astype('int32') #PD = 1, SNP = 0

    return Output #Datset in alphabetical order, NaNs removed and encoded PD label

### Split dataset into training and validation sets

In [62]:
def Train_Test_Split(Output):
    """      
    Input:      Output           Cleaned dataframe

    Returns:    Training_Set     80% training data
                Testing_Set      20% testing data

    80% training and 20% testing split. Outputs the data to files. Splits are randomly shuffled
    """
    Training_Set, Testing_Set = train_test_split(Output,train_size = 0.8)
    labels = Training_Set['dataset_pd'].astype('int32')
    
    
    with open('Training dataset.txt', 'w') as file: #Writes training features to text file
        file.write(Training_Set.to_string())
    with open('Testing dataset.txt', 'w') as file: #Writes training class labels to text file
        file.write(Testing_Set.to_string())


    return Training_Set, Testing_Set, labels

### Initial evaluation

In [5]:
# def test(Initial_RFC, Input_test, Classes_test):
#     """ Input:  Input_test      Features test data
#                 Classes_test    Class label test data

#         Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
#         are the class labels testing data
#     """

#     Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
#     print(f"              **Initial Evaluation**\n")
#     print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
#     print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


### Balancing

In [57]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
    
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [58]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [59]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [100]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   Input_balance  Array of balanced input training data
                       Label_balance  Array of balanced classes assigned to training data
        Create arrays for the input training data and classes, as needed for predicting the probability.
        The index [i] is the identifier between the two arrays
    """
    Input_balance = []
    Label_balance = []
    for i in range(len(inData)):
        if usedLines[i]:
            Input_balance.append(inData[i])
            Label_balance.append(classData[i])
            
    Input_balance = np.stack(Input_balance, axis =0)
    Label_balance = np.stack(Label_balance, axis =0)
    
    return Input_balance, Label_balance

### Balance for n folds

In [101]:
def Balance_Folds(BF, usedLines, Input_balance, Label_balance):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    Input_balance     Input_balance  Array of balanced input training data
                    Label_balance     Array of balanced classes assigned to training data

        Returns:    Input_folds       List of 5 balanced arrays of training data
                    Output_folds      List of 5 balanced arrays of training data's labels

        Perform the balance_data() function n number of balancing fold times. Return lists for training data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    for fold in range(BF):
        Input_folds.append(Input_balance)
        Output_folds.append(Label_balance)


    with open('Balanced training data.txt', 'w') as f:
        for number, fold in zip(range(BF), Input_folds):
            f.write(f"Fold: {number}\n\n{fold}\n\n\n")
        
    return Input_folds, Output_folds

In [102]:
file                         = 'E2.csv'
Output                       = Clean_data(file)
Training_Set, Testing_Set, labels    = Train_Test_Split(Output)

inData                       = pd.DataFrame(Training_Set).to_numpy()
classData                    = pd.DataFrame(labels).to_numpy()

minClass, minSize, maxSize   = find_minority_class(classData)
BF                           = Balance_ratio(maxSize, minSize)
usedLines                    = balance(inData, classData, minClass, minSize)

Input_balance, Label_balance = balance_data(inData, classData, usedLines)
Input_folds, Output_folds    = Balance_Folds(BF, usedLines, Input_balance, Label_balance)


In [115]:
Input_folds[0][:,0]

array(['2r3v:A:52:N', '3bpt:A:122:C', '1qki:A:454:H', '2d1k:C:725:A',
       '1x0x:A:54:V', '3c2i:A:101:T', '3b63:B:108:T', '1m4u:A:217:G',
       '1d5r:A:173:C', '3e4e:A:457:L', '2pm8:A:100:S', '2fy3:A:187:T',
       '3u2o:A:285:I', '3k8o:E:51:S', '1ijq:A:525:D', '2j6l:A:413:A',
       '1sir:A:249:T', '1bhg:A:382:H', '1z83:A:123:Q', '1uwy:A:7:H',
       '3mdj:B:346:D', '2p0r:A:277:W', '1dtw:A:245:R', '1ht0:A:47:H',
       '3msx:B:522:T', '3odq:A:48:R', '2dnf:A:28:A', '1ey2:A:62:C',
       '2zw3:A:160:S', '3odq:A:27:G', '2vxp:A:540:S', '1nuf:A:653:R',
       '1ya4:A:199:H', '1aut:C:48:Y', '2yhd:A:812:F', '3pdf:A:423:G',
       '2xwd:A:456:P', '2ppl:A:271:V', '3bu3:A:1068:E', '2gzk:A:46:H',
       '1shr:B:10:D', '1fqy:A:165:D', '1ijq:A:402:T', '1bqq:M:273:N',
       '1bp1:A:249:V', '1xmj:A:579:G', '3ee6:A:206:C', '2yrg:A:34:F',
       '2hi9:A:86:E', '2xwd:A:391:L', '1d7p:M:2205:L', '3sza:A:308:E',
       '3m1n:B:111:N', '2i3y:A:85:P', '1jbq:A:353:M', '1shr:B:2:L',
       '3qrf:F:371:C',

### RFC hyperparameter tuning

In [13]:
#  def Hyperparameter(BF, Input_folds, Output_folds):
#     """ Input:      BF                Number of balancing folds needed
#                     Input_folds       List of 5 balanced arrays of training data
#                     Output_folds      List of 5 balanced arrays of training data's labels

#         Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

#         Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
#     """  
#     estimator = RandomForestClassifier()
#     param_grid = {
#                 'n_estimators':np.arange(50,500,50),
#                 'max_depth': np.arange(2, 10, 2),
#                 'max_samples': np.arange(0.2, 1.2, 0.2)
#                   }
#     BF_RFC_HP = []

#     for i in range(BF):
#         HPtuning = RandomizedSearchCV(
#             estimator,
#             param_grid, 
#             scoring = 'balanced_accuracy',
#             cv = 10,
#             n_jobs = 6, #how many cores to run in parallel
#             verbose = 2
#             ).fit(Input_folds[i], Output_folds[i].ravel())
#         BF_RFC_HP.append(HPtuning.best_params_)
    
#     return(BF_RFC_HP)

In [6]:
def Group_data(Training_Set):
    """      
    Input:      Training_Set     80% training dataset

    Returns:    Input_CV         Datastet with input features for training
                Output_CV        Datastet with class labels for training
                Protein_Groups   List of proteins for grouping

    Alphabetically order the identifier column, extract only the 'pdbcode' and make that the identifer column.
    Use this formatted dataset to create the training features, class labels, and group identifiers.
    """
    Group_df = Training_Set
    Group_df.sort_values(by=['pdbcode:chain:resnum:mutation'], inplace = True)

    PDB_codes = []
    for i in range(len(Group_df)): 
        PDB_codes.append(Group_df.iloc[i][0].partition(':')[0])

    Group_df.insert(0, 'PDB code', PDB_codes)
    Group_df.drop(['pdbcode:chain:resnum:mutation'], axis=1, inplace=True) #Remove 'pdbcode:chain:resnum:mutation' column
    Group_df.reset_index(inplace = True, drop = True)
    
    Input_CV = Group_df.drop(['dataset_pd'], axis =1)
    Output_CV = Group_df['dataset_pd'].copy().astype('int32') 
    Protein_Groups = Group_df['PDB code'].to_list()

    return Input_CV, Output_CV, Protein_Groups

### Group K-fold CV

In [28]:
def CV(Input_CV, Output_CV, Protein_Groups):
    """      
    Input:      Input_CV             Datastet with input features for training
                Output_CV            Datastet with class labels for training
                Protein_Groups       List of proteins for grouping
            
    Returns:    Input_train_list     List of training features, for each fold
                Classes_train_list   List of training classes, for each fold
                Input_val_list       List of validating features , for each fold
                Classes_val_list     List of validating classes , for each fold

    Stratified K-fold CV that maintains protein groups, attempts to preserve number of samples of each class 
    for each fold, and ensures protein groups are separated. 5 folds.
    """
    CV = StratifiedGroupKFold(n_splits = 5, shuffle = True) #Only shuffles proteins in each group, not groups in fold
    
    Fold = 0
    Input_train_list = []
    Classes_train_list = []
    Input_val_list = []
    Classes_val_list = []
    
    for train_idx, val_idx in CV.split(Input_CV, Output_CV, Protein_Groups):
        Rd = np.random.randint(time.time()) #Random number from 1 to time since epoch
        Input_train = Input_CV.loc[train_idx]
        Input_train.drop(['PDB code'], inplace = True)
        Classes_train = Output_CV.loc[train_idx]
        Classes_train.drop(['PDB code'], inplace = True)

        
        Input_train_list.append(Input_train.sample(frac=1, random_state=Rd)) #shuffles each split and adds to list.
                                                                             # Random state so train and test data index same
        Classes_train_list.append(Classes_train.sample(frac=1, random_state=Rd))

        Input_val = Input_CV.loc[val_idx]
        Classes_val = Output_CV.loc[val_idx]
        
        Input_val_list.append(Input_val.sample(frac=1, random_state=Rd))
        Classes_val_list.append(Classes_train.sample(frac=1, random_state=Rd))


    return(Input_train_list,Classes_train_list,Input_val_list,Classes_val_list)


### Train RFC on balanced dataset

In [21]:
def BF_training(BF, Input_folds, Output_folds): 
    """ Input:      BF              Number of balancing folds
                    Input_folds     List of 5 balanced arrays for training data
                    Output_folds    List of 5 balanced arrays of training data's labels

        Returns:    BF_RFC          List of RFC's trained on data in each balancing fold

        Create a model that returns probability predictions for each fold, using Balance_Fold() as input
    """    
    BF_RFC = []
    
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(
                                             verbose = 1
                                            )) #Generates a RF for each fold 
        BF_RFC[i].fit(Input_folds[i], Output_folds[i].ravel()) #Fits the RFC to balanced training data    
        
    return BF_RFC

#### Test RFC on test set

In [22]:
def BFC_test(BF_RFC, Input_test):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_test      20% unseen testing data split before the balancing folds
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    Prob_matrix = [] #Empty list
    Prob_matrixlist = []
    for i in range(len(BF_RFC)):
        Prob_list = BF_RFC[i].predict_proba(Input_test.values)
        Prob_matrix.append(Prob_list)   
        
    with open('Test probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

### Weighted voting

In [23]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = [] #Empty list
    Sc_SNP = [] #Empty list
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold
    

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     

    Vote_arr = [] #Empty list
    
    for i in range(len(Classes_test)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list
            
    Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array, shape (674,1)
    Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array
        
    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


### Final confidence

In [24]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out        Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [25]:
def evalutation(Classes_test, Final_vote, S_Out):
    """ Input:      Classes_test       Class label test data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on test data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
    print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")
    
    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )
    

### Main Program

In [29]:
file                         = 'E2.csv'
Output                       = Clean_data(file)
Training_Set, Testing_Set    = train(Output)
Input_CV, Output_CV, Protein_Groups = Group_data(Training_Set)

Input_train_list, Classes_train_list, Input_val_list, Classes_val_list = CV(Input_CV, Output_CV, Protein_Groups)

inData                       = pd.DataFrame(Input_train_list[0]).to_numpy()
classData                    = pd.DataFrame(Classes_train_list[0]).to_numpy()

minClass, minSize, maxSize   = find_minority_class(classData)
BF                           = Balance_ratio(maxSize, minSize)
usedLines                    = balance(inData, classData, minClass, minSize)

Input_balance, Label_balance = balance_data(inData, classData, usedLines)
Input_folds, Output_folds    = Balance_Folds(BF, usedLines, Input_balance, Label_balance)

# BF_RFC_HP                    = Hyperparameter(BF, Input_folds, Output_folds)
BF_RFC                       = BF_training(BF, Input_folds, Output_folds)
Prob_matrix                  = BFC_test(BF_RFC, Input_test)

Final_vote, Sum_PD, Sum_SNP  = Weighted_Vote(Prob_matrix, BF)
S_Out                        = Final_score(Sum_PD, Sum_SNP, BF)

evalutation(Classes_test, Final_vote,S_Out)


KeyError: "['PDB code'] not found in axis"